In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#Neural Network with TensorFlow and KERAS
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout,Conv2D
# all  related libaray upload in  the given 
#model library 
from tensorflow.keras.models import Sequential
#activation function 
from tensorflow.keras.activations import softmax,relu,sigmoid
#layer 
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPooling2D,Dropout,Conv2D
#losss 
from tensorflow.keras.losses import categorical_crossentropy,binary_crossentropy
#optimizer 
from tensorflow.keras.optimizers import SGD,Adagrad,Adam,Adamax
#calll back and checkpoint 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
plant=pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
plant

In [ ]:
plant.labels.value_counts()

In [ ]:
plt.figure(figsize=(12, 6))
plant.labels.value_counts().plot.bar()
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import vgg16, VGG16

In [ ]:
idg=tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, rotation_range=30,
                                                    rescale=1/255,validation_split=0.3,
                                                    preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

In [ ]:
#train 

train_idg= idg.flow_from_dataframe(plant,directory="../input/plant-pathology-2021-fgvc8/train_images",x_col='image',y_col='labels',target_size=(224,224),batch_size=128,subset= "training")

#validation 

val_idg= idg.flow_from_dataframe(plant,directory="../input/plant-pathology-2021-fgvc8/train_images",x_col='image',y_col='labels',target_size=(224,224),batch_size=128,subset="validation")

In [ ]:
#download the vgg model 

vggmodel=tf.keras.applications.vgg16.VGG16(include_top=False,input_shape=(224,224,3))

In [ ]:
for layer in vggmodel.layers:
  layer.trainable=False

In [ ]:
for layer in vggmodel.layers:
  print(layer.name,layer.trainable)

In [ ]:
#functional way two build the the model API

vgg_output=vggmodel.output
flatten_output=tf.keras.layers.Flatten()(vgg_output)
dense1=tf.keras.layers.Dense(256,activation='relu')(flatten_output)
bn1=tf.keras.layers.BatchNormalization()(dense1)
dp1=tf.keras.layers.Dropout(0.3)(bn1)
dense2=tf.keras.layers.Dense(128,activation='relu')(dp1)
dp2=tf.keras.layers.Dropout(0.2)(dense2)
dense3=tf.keras.layers.Dense(12,activation='softmax')(dp2)


In [ ]:
finalModel=tf.keras.models.Model(inputs=[vggmodel.input],outputs=[dense3])

In [ ]:
#final model 

finalModel.summary()

In [ ]:
#compile the model 

finalModel.compile(optimizer=tf.keras.optimizers.SGD(),loss=tf.keras.losses.categorical_crossentropy,metrics=['acc'])

In [ ]:
finalModel_history=finalModel.fit(train_idg,epochs=50,validation_data=val_idg)